In [121]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [122]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'data/train.csv.zip' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
tX_2 = load_csv_data_2(DATA_TRAIN_PATH)
print(np.shape(tX_2))

(250000, 19)


# Data cleaning and normalisation

In [123]:
# we know that very low values are used to signal unavailable data
tX[tX < -900] = np.nan

In [124]:
tX_mean = np.nanmean(tX,axis=0)

tX_std = np.nanstd(tX,axis=0)
norm_tX = np.subtract(tX, tX_mean, where=np.isfinite(tX_mean))
norm_tX = np.divide(norm_tX, tX_std, where=tX_std>0)
norm_tX[np.isnan(norm_tX)] =0
print(norm_tX)

#tX2_mean = np.nanmean(tX_2,axis=0)
#tX2_std = np.nanstd(tX_2,axis=0)
#norm_tX2 = np.subtract(tX_2, tX2_mean, where=np.isfinite(tX2_mean))
#norm_tX2 = np.divide(norm_tX2, tX2_std, where=tX2_std>0)
#norm_tX = np.c_[np.ones(num_samples), norm_tX] car je le fais dans build poly

[[ 0.28991353  0.06833197  0.40768027 ...  0.61614788 -1.36131161
   0.4125105 ]
 [ 0.68202131  0.55250482  0.54013641 ...  0.          0.
  -0.27381996]
 [ 0.          3.19515553  1.09655998 ...  0.          0.
  -0.29396985]
 ...
 [-0.28624947  0.31931645 -0.13086367 ...  0.          0.
  -0.31701723]
 [-0.46960659 -0.84532397 -0.30297338 ...  0.          0.
  -0.74543941]
 [ 0.          0.66533608 -0.25352276 ...  0.          0.
  -0.74543941]]


# Least squares and Polynomial regression


In [125]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    a = tx.T.dot(tx)
    b = tx.T.dot(y)
    
    
    return np.linalg.solve(a, b)

In [126]:
from costs import compute_mse
from plots import *

def polynomial_regression(x):
    """Constructing the polynomial basis function expansion of the data,
       and then running least squares regression."""
    # define parameters
    degrees = [1,3,7,12]
    
    
    for ind, degree in enumerate(degrees):
        # form dataset to do polynomial regression.
        tx = build_poly(x, degree)
        
        # least squares
        weights = least_squares(y, tx)
        
        
        # compute RMSE
        rmse = np.sqrt(2 * compute_mse(y, tx, weights))
        print("Processing {i}th experiment, degree={d}, rmse={loss}".format(
              i=ind + 1, d=degree, loss=rmse))
       
        # print
        #print(np.shape(y))
        #print(np.shape(tx))
        #print(np.shape(weights))
        
    return weights
                


In [127]:

#weights = polynomial_regression(norm_tX)
#weights = polynomial_regression(norm_tX2)


In [128]:
def split_data(x, y, ratio, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [131]:
def train_test_split_demo(x, y, degree, ratio, seed):
    """polynomial regression with different split ratios and different degrees."""
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)
    # form tx
    tx_tr = build_poly(x_tr, degree)
    tx_te = build_poly(x_te, degree)

    weight = least_squares(y_tr, tx_tr)
    
    #if ((ratio == 0,9) and (degree == 7)):
    weights = weight
    #print(weights)

    # calculate RMSE for train and test data.
    rmse_tr = np.sqrt(2 * compute_mse(y_tr, tx_tr, weight))
    rmse_te = np.sqrt(2 * compute_mse(y_te, tx_te, weight))

    print("proportion={p}, degree={d}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
          p=ratio, d=degree, tr=rmse_tr, te=rmse_te))
    return weight

In [132]:
seed = 6
degrees = [7]
split_ratios = [0.9]
weights = train_test_split_demo(norm_tX, y, degree, split_ratio, seed)
#print(weights)

#for split_ratio in split_ratios:
    #for degree in degrees:
      #  train_test_split_demo(norm_tX, y, degree, split_ratio, seed)
        
#for split_ratio in split_ratios:
 #   for degree in degrees:
  #      train_test_split_demo(norm_tX2, y, degree, split_ratio, seed)

[-1.63846625e+00  4.32972961e-01 -3.90281463e-01 -3.58878001e-01
  7.05052572e-02  2.36840263e-01 -2.30021948e-01 -1.88355950e-01
  1.94357202e-01 -2.39445474e-02 -2.24144495e+02 -1.96113958e-01
 -1.50648176e-01  1.96666666e-01  4.36111359e+01  9.29958904e-03
 -1.28202227e-03  4.29794961e+01  5.71082946e-03  3.59189189e-03
 -2.40448242e-03  7.76453256e-03 -2.62721400e-02  7.01847564e+01
  1.56659322e-01 -7.18048227e-05 -1.13590486e-02  1.85668930e-01
  3.01843356e-03  2.28393072e-02  1.89775855e+02 -1.85806326e-01
  8.90463250e-03 -4.28457037e-02  8.22522285e-02  1.02069330e-01
  3.60599648e-01 -2.98687526e-02 -4.70943523e-03 -1.06756105e-02
  4.48467102e-02  1.12425379e-01  2.79261549e-02  3.62528573e-02
 -9.61560662e-02  1.21498498e-02 -6.01680381e-03 -1.13801127e-01
 -4.71817415e-02 -1.96036514e-03  4.11082600e-02 -6.16966284e-03
 -7.40118812e-02  2.75143493e+01 -1.01566080e-01  1.63819671e-01
  1.04899949e-02 -6.28894701e-02  2.22402359e-01 -4.89182787e-02
 -2.48429160e-02 -9.31168

## Generate predictions and save ouput in csv format for submission:

In [133]:
DATA_TEST_PATH = 'data/test.csv.zip' # TODO: download train data and supply path here 
y, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [134]:
# we know that very low values are used to signal unavailable data
#tX_test[tX_test < -900] = np.nan

In [137]:
tX_test_mean = np.nanmean(tX_test,axis=0)
tX_test_std = np.nanstd(tX_test,axis=0)
norm_tX_test = np.subtract(tX_test, tX_test_mean, where=np.isfinite(tX_test_mean))
norm_tX_test = np.divide(norm_tX_test, tX_test_std, where=tX_test_std>0)
norm_tX_test = build_poly(norm_tX_test, 7)


In [138]:
OUTPUT_PATH = 'data/sample-submission' # TODO: fill in desired name of output file for submission
#print(np.shape(y))
#print(np.shape(norm_tX_test))
#print(np.shape(weights))
print(weights)
y_pred = predict_labels(weights, norm_tX_test)

create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

[-1.63846625e+00  4.32972961e-01 -3.90281463e-01 -3.58878001e-01
  7.05052572e-02  2.36840263e-01 -2.30021948e-01 -1.88355950e-01
  1.94357202e-01 -2.39445474e-02 -2.24144495e+02 -1.96113958e-01
 -1.50648176e-01  1.96666666e-01  4.36111359e+01  9.29958904e-03
 -1.28202227e-03  4.29794961e+01  5.71082946e-03  3.59189189e-03
 -2.40448242e-03  7.76453256e-03 -2.62721400e-02  7.01847564e+01
  1.56659322e-01 -7.18048227e-05 -1.13590486e-02  1.85668930e-01
  3.01843356e-03  2.28393072e-02  1.89775855e+02 -1.85806326e-01
  8.90463250e-03 -4.28457037e-02  8.22522285e-02  1.02069330e-01
  3.60599648e-01 -2.98687526e-02 -4.70943523e-03 -1.06756105e-02
  4.48467102e-02  1.12425379e-01  2.79261549e-02  3.62528573e-02
 -9.61560662e-02  1.21498498e-02 -6.01680381e-03 -1.13801127e-01
 -4.71817415e-02 -1.96036514e-03  4.11082600e-02 -6.16966284e-03
 -7.40118812e-02  2.75143493e+01 -1.01566080e-01  1.63819671e-01
  1.04899949e-02 -6.28894701e-02  2.22402359e-01 -4.89182787e-02
 -2.48429160e-02 -9.31168